In [1]:
# libraries

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import ast


2025-01-23 11:09:56.269528: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
df = pd.read_csv("large_board_dataset2.csv")

In [38]:
df.head()
df["x"] = df["x"].apply(lambda x: np.array(ast.literal_eval(x)).reshape(2, 6, 7))

In [39]:
def preprocess_board(board):
    if board["whose_turn"] != "red":
        # Flip the layers to make each board look like it's from the perspective of "red", aka plus
        board["x"] = board["x"][::-1, :, :]
    return board

In [40]:
df = df.apply(preprocess_board, axis=1)

In [41]:
x = df["x"]
y = df["y"]

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

In [75]:
cnn = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=4,kernel_size=(2,2),activation=tf.nn.relu,input_shape=(6, 7, 2)),
        # had to adjust data_format because it was treating the images as 2x6 with 7 channels
        tf.keras.layers.MaxPooling2D(pool_size = (2,2),strides=(1, 1), padding = "same"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation=tf.nn.relu,kernel_regularizer = tf.keras.regularizers.l2(0.0002)),
        tf.keras.layers.Dense(64,activation=tf.nn.relu,kernel_regularizer = tf.keras.regularizers.l2(0.0005)),
        tf.keras.layers.Dense(7,activation=tf.nn.softmax)
        ])

In [51]:
X_train = np.stack(X_train)  # Stack list of arrays into a single numpy array
X_test = np.stack(X_test)
y_train = np.array(y_train)  # Convert labels to numpy array
y_test = np.array(y_test)

In [71]:
X_train = X_train.transpose(0, 2, 3, 1)  # Convert (num_samples, 2, 6, 7) to (num_samples, 6, 7, 2)
X_train = X_train.astype('float32')      # Ensure correct dtype

In [76]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

cnn.summary()

cnn.fit(X_train,y_train,epochs=200,validation_split=0.2,batch_size=100)

pred_probs = cnn.predict(X_test)
pred = np.argmax(pred_probs, axis=1)
print(np.mean(pred==y_test))

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 5, 6, 4)        │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 5, 6, 4)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 128)            │        15,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,235 (94.67 KB)

 Trainable params: 24,235 (94.67 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.2200 - loss: 1.9357 - val_accuracy: 0.2929 - val_loss: 1.8257
Epoch 2/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3103 - loss: 1.7880 - val_accuracy: 0.3526 - val_loss: 1.7035
Epoch 3/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3622 - loss: 1.6790 - val_accuracy: 0.3774 - val_loss: 1.6423
Epoch 4/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3936 - loss: 1.6153 - val_accuracy: 0.3839 - val_loss: 1.6321
Epoch 5/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4039 - loss: 1.5880 - val_accuracy: 0.3926 - val_loss: 1.6015
Epoch 6/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4197 - loss: 1.5505 - val_accuracy: 0.3987 - val_loss: 1.5860
Epoch 7/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4280 - loss: 1.5286 - val_accuracy: 0.4051 - val_loss: 1.5770
Epoch 8/200
418/418 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4338 - loss: 1.5137 - val_accu

KeyboardInterrupt: 

In [49]:
print(f"X_train dtype: {X_train.dtype}, shape: {X_train.shape}")


X_train dtype: object, shape: (52238,)


In [56]:
len(df)

65298